In [1]:
doc = {
    "id": 1,
    "first_name": "Celestia",
    "last_name": "McCathie",
    "email": "cmccathie0@topsy.com"
}

In [2]:
mappings = {
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    },
    "mappings":{
        "properties": {
            "id": {
                "type": "integer"
            },
            "first_name": {
                "type": "keyword"
            },
            "last_name": {
                "type": "keyword"
            },
            "email": {
                "type": "keyword"
            }
        }
    }
        
}

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch(hosts='127.0.0.1', port=9200)
es.indices.delete(index='test')
# 設定mapping
es.indices.create(index="test", body=mappings)
# index一筆document
res = es.index(index="test", id=1, body=doc)

In [9]:
query = {
    "query": {
        "bool": {
            "must": [
                { "match": { "first_name": "Celestia" }},
                { "match": { "last_name": "McCathie" }}
            ]
        }
    }
}

res = es.search(index="test", body=query)
print(res['hits'])

{'total': {'value': 1, 'relation': 'eq'}, 'max_score': 0.5753642, 'hits': [{'_index': 'test', '_type': '_doc', '_id': '1', '_score': 0.5753642, '_source': {'id': 1, 'first_name': 'Celestia', 'last_name': 'McCathie', 'email': 'cmccathie0@topsy.com'}}]}


In [12]:
sql = """
select * 
from test 
"""

res = es.sql.query(body={'query': sql})
print(res['rows'])

[['cmccathie0@topsy.com', 'Celestia', 1, 'McCathie']]


### 自行生成資料

In [15]:
mappings = {
    "settings": {
        "number_of_shards": 2,
        "number_of_replicas": 1
    },
    "mappings":{
        "properties": {
            "id": {
                "type": "integer"
            },
            "first_name": {
                "type": "keyword"
            },
            "last_name": {
                "type": "keyword"
            },
            "email": {
                "type": "keyword"
            },
            "gender": {
                "type": "keyword"
            },
            "ip_address": {
                "type": "keyword"
            }
        }
    }
        
}

In [16]:
import json 
with open('MOCK_DATA.json') as f:
    data = json.load(f)
data


[{'id': 1,
  'first_name': 'Phoebe',
  'last_name': 'Spikings',
  'email': 'pspikings0@smugmug.com',
  'gender': 'Genderqueer',
  'ip_address': '96.59.191.100'},
 {'id': 2,
  'first_name': 'Robinia',
  'last_name': 'MacKee',
  'email': 'rmackee1@nasa.gov',
  'gender': 'Agender',
  'ip_address': '126.140.137.36'},
 {'id': 3,
  'first_name': 'Selle',
  'last_name': 'Redholes',
  'email': 'sredholes2@youku.com',
  'gender': 'Polygender',
  'ip_address': '23.170.147.188'},
 {'id': 4,
  'first_name': 'Hoebart',
  'last_name': 'Hawkett',
  'email': 'hhawkett3@miitbeian.gov.cn',
  'gender': 'Non-binary',
  'ip_address': '252.219.56.247'},
 {'id': 5,
  'first_name': 'Celie',
  'last_name': 'Cleevely',
  'email': 'ccleevely4@blogtalkradio.com',
  'gender': 'Male',
  'ip_address': '26.204.228.63'},
 {'id': 6,
  'first_name': 'Allen',
  'last_name': 'Laffranconi',
  'email': 'alaffranconi5@statcounter.com',
  'gender': 'Genderfluid',
  'ip_address': '100.229.117.133'},
 {'id': 7,
  'first_name': 

In [19]:
# len(data)
data[0]

{'id': 1,
 'first_name': 'Phoebe',
 'last_name': 'Spikings',
 'email': 'pspikings0@smugmug.com',
 'gender': 'Genderqueer',
 'ip_address': '96.59.191.100'}

In [21]:
es.indices.delete(index='hw')
es.indices.create(index="hw", body=mappings)
for i in range(len(data)):
    es.index(index="hw", id=i, body=data[i])

In [26]:
query = {
    "query": {
        "bool": {
            "must": [
                { "match": { "gender": "Female" },
                  "match": {'first_name': 'Rubin'}}
            ]
        }
    }
}

res = es.search(index="hw", body=query)
print(res['hits'])

{'total': {'value': 1, 'relation': 'eq'}, 'max_score': 5.7493925, 'hits': [{'_index': 'hw', '_type': '_doc', '_id': '58', '_score': 5.7493925, '_source': {'id': 59, 'first_name': 'Rubin', 'last_name': 'Heimann', 'email': 'rheimann1m@eepurl.com', 'gender': 'Female', 'ip_address': '123.133.253.102'}}]}


In [27]:
sql = """select * from hw where gender='Female' and first_name='Rubin'"""

res = es.sql.query(body={'query': sql})
print(res['rows'])

[['rheimann1m@eepurl.com', 'Rubin', 'Female', 59, '123.133.253.102', 'Heimann']]
